# Airbnb Model Evaluation

In [1]:
#importing packages
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, NaiveBayes, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SparkSession
import pandas as pd

### Loading Data

In [2]:
# Start spark session
spark = SparkSession.builder.getOrCreate()

In [3]:
# Read data in json format
train = spark.read.option("header",True).parquet("./data_preprocessed/train_data.parquet")
test = spark.read.option("header",True).parquet("./data_preprocessed/test_data.parquet")

### Loading Models

In [4]:
model_ridge = PipelineModel.load('./models/ridge.model')
model_lasso = PipelineModel.load('./models/lasso.model')
model_bayes = PipelineModel.load('./models/naive_bayes.model')
model_rf = PipelineModel.load('./models/random_forest.model')
model_gbt = PipelineModel.load('./models/gbt.model')

### AUC Table

In [5]:
models = [
    model_ridge,
    model_lasso,
    model_bayes,
    model_rf,
    model_gbt
]

ClassEval = BinaryClassificationEvaluator()
OtherClassEval = MulticlassClassificationEvaluator()

training_auc = [ClassEval.evaluate(model.transform(train)) for model in models]
test_auc = [ClassEval.evaluate(model.transform(test)) for model in models]

training_acc = [OtherClassEval.evaluate(model.transform(train), {OtherClassEval.metricName: "accuracy"}) for model in models]
test_acc = [OtherClassEval.evaluate(model.transform(test), {OtherClassEval.metricName: "accuracy"}) for model in models]

training_f1 = [OtherClassEval.evaluate(model.transform(train), {OtherClassEval.metricName: "f1"}) for model in models]
test_f1 = [OtherClassEval.evaluate(model.transform(test), {OtherClassEval.metricName: "f1"}) for model in models]

pd.DataFrame({
    "Model_Name" : [
        "Ridge Logistic Regression",
        "Lasso Logistic Regression",
        "Naive Bayes",
        "Random Forest",
        "Gradient Boosted Tree"
    ],
    'Training Accuracy' : training_acc,
    'Test Accuracy' : test_acc,
    'Training AUC' : training_auc,
    'Test AUC' : test_auc,
    'Training F1' : training_f1,
    'Test F1' : test_f1
}).sort_values('Test AUC', ascending = False).reset_index().drop(columns = 'index')

,Model_Name,Training Accuracy,Test Accuracy,Training AUC,Test AUC,Training F1,Test F1
0,Gradient Boosted Tree,0.691129,0.692706,0.741573,0.739274,0.683501,0.685122
1,Random Forest,0.687851,0.689469,0.732884,0.732937,0.677851,0.679404
2,Lasso Logistic Regression,0.684212,0.686811,0.721210,0.720946,0.672694,0.675218
3,Ridge Logistic Regression,0.684212,0.686811,0.721210,0.720943,0.672694,0.675218
4,Naive Bayes,0.618926,0.618028,0.674520,0.675060,0.618922,0.618067


From our table, it seems that overall both models performed around the same on the training and test set which is likely because we used crossvalidation.

We found that the highest performing model in this case is the Gradient Boosted Tree model with a test AUC of 0.73.